# Example workflow

### Import Packages

In [1]:
import rectanglepy as rectangle
from anndata import AnnData

## Load tutorial data

In [2]:
sc_counts, annotations, bulks  = rectangle.pp.load_tutorial_sc_data()

In [3]:
sc_counts.head()

,E2L4_GATGCTACAGGCACAA,L3_AACGTCACATCAGCAT,L3_GGAATGGGTCTAACGT,E2L7_GTCATTTAGGCCTTCG,E2L6_CCCTTAGCATTGACCA,E2L8_CTGAATGAGACGTCGA,E2L2_AAGCATCAGAAGATCT,E2L2_ACTTCCGTCTGGACCG,E2L6_GTCGTTCGTGTTACAC,E2L6_TTACCGCAGGTAGCCA,...,E2L2_CGAGTGCAGCGCATCC,L2_CATGCGGCAATCGCGC,E2L1_CGGACACAGACTAGAT,L5_AGAAGCGCACGACAGA,L3_CATTGCCCAAGACAAT,E2L8_CCTTGTGTCACTCACC,E2L4_CTAGACACAGGACTAG,E2L5_GGTTGTACAGAATCGG,E2L5_ATGGTTGAGTGAGGTC,E2L3_GCTTTCGTCGACGATT
NOC2L,0,0,0,2,0,0,1,0,0,0,...,3,1,0,0,1,0,2,1,1,1
HES4,11,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ISG15,7,1,0,1,0,0,4,0,0,1,...,13,1,0,5,0,2,0,4,4,3
SDF4,0,0,2,2,1,1,2,0,0,0,...,5,3,4,2,1,0,1,0,2,0
B3GALT6,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0


In [4]:
annotations.head()

E2L4_GATGCTACAGGCACAA    Monocytes
L3_AACGTCACATCAGCAT      Monocytes
L3_GGAATGGGTCTAACGT      Monocytes
E2L7_GTCATTTAGGCCTTCG    Monocytes
E2L6_CCCTTAGCATTGACCA    Monocytes
Name: 0, dtype: object

In [5]:
bulks.head()

,pbmc_1,pbmc_10,pbmc_12,pbmc_2,pbmc_4,pbmc_5,pbmc_6,pbmc_7,pbmc_9
ACO2,61.622872,50.203845,52.941600,35.418700,32.456407,49.126300,53.256380,45.731200,46.525152
AFF3,3.295273,3.252860,2.434910,1.679995,7.960524,7.122442,2.090572,3.170789,2.467964
AIF1,597.287900,828.944100,660.822400,630.792900,511.570690,583.109220,706.516940,880.898400,798.426800
AKNA,77.983332,61.366859,78.963328,76.144263,170.505107,161.732441,67.800729,84.280110,62.216777
ARF4,61.971550,44.870757,63.573104,37.797272,37.027217,45.130400,62.890100,58.591060,50.318090


### Convert to AnnData object

In [6]:
sc_adata = AnnData(sc_counts.T, obs=annotations.to_frame(name="cell_type"))
bulks_adata = AnnData(bulks.T, obs=bulks.columns.to_frame(name="bulk"))

## Create Signature result

In [ ]:
signature_result = rectangle.pp.build_rectangle_signatures(sc_adata)

This creates a {class}`rectanglepy.pp.RectangleSignatureResult` object.

## Deconvolute bulk data

In [12]:
estimations = rectangle.tl.deconvolution(signature_result, bulks_adata)

2023-11-21 12:32:37.365 | INFO     | rectanglepy.tl.deconvolution:deconvolution:178 - Percentage of genes in the signature that are in bulk is 77.78%
2023-11-21 12:32:38.426 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 0
2023-11-21 12:32:39.309 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 2
2023-11-21 12:32:39.311 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 6
2023-11-21 12:32:39.313 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 7
2023-11-21 12:32:39.317 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 5
2023-11-21 12:32:39.321 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 3
2023-11-21 12:32:39.326 | INFO     | rectanglepy.tl.deconvolution:_process_bulk:197 - Deconvolute fractions for bulk: 1
2023-11-21

In [13]:
estimations

,B cells,Monocytes,NK cells,Plasma cells,T cells CD4 conv,T cells CD8,Tregs,mDC,pDC,Unknown
pbmc_1,0.024612,0.342098,0.052341,5.695104e-03,0.142145,0.283058,3.798142e-02,0.103029,0.009041,0.000000
pbmc_10,0.017537,0.382264,0.027946,2.853735e-04,0.007032,0.225360,1.399217e-01,0.156580,0.043074,0.000000
pbmc_12,0.016941,0.118410,0.111627,0.000000e+00,0.019852,0.423309,4.177850e-03,0.230941,0.033572,0.041170
pbmc_2,0.001540,0.274858,0.052025,0.000000e+00,0.164213,0.278624,-2.115860e-17,0.198859,0.029880,0.000000
pbmc_4,0.038034,0.153828,0.138206,4.927018e-03,0.123980,0.352626,4.161966e-02,0.102410,0.044369,0.000000
pbmc_5,0.084280,0.238558,0.082768,0.000000e+00,0.000581,0.409376,2.996198e-20,0.157283,0.027154,0.000000
pbmc_6,0.016412,0.467883,0.050700,1.307287e-02,0.006617,0.370235,2.652160e-18,0.063343,0.011738,0.000000
pbmc_7,0.005263,0.143307,0.144074,0.000000e+00,0.023120,0.428678,-9.168158e-18,0.209814,0.025695,0.020049
pbmc_9,0.000002,0.254132,0.118803,4.039122e-22,0.006600,0.328968,-4.042499e-18,0.217132,0.045488,0.028874
